In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib as mpl

from scipy.stats import bernoulli
from methods import * 

%load_ext autoreload
%autoreload 2

In [ ]:
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['text.usetex'] = False

# Simulated Data - Figure 1

In [ ]:
def plot_wealth(arr, label, c): 
    """Simple helper to plot given array with std"""
    
    mean = np.mean(arr, axis=0)   
    std = np.std(arr, axis=0)
    plt.plot(range(size), mean, label=label, c=c)
    plt.fill_between(range(len(mean)), mean+std, mean-std, alpha=0.1, color=c)
    


In [ ]:
mpl.rcParams['font.size'] = 14

size = 300
iters = 100 


huge_gap = []
big_gap = []
med_gap = []
no_gap = []

for _ in range(iters): 
    
    
    X1 = bernoulli.rvs(0.7, size=size)
    X2 = bernoulli.rvs(0.4, size=size)
    X3 = bernoulli.rvs(0.4, size=size)
    X4 = bernoulli.rvs(0.55, size=size)
    X5 = bernoulli.rvs(0.9, size=size)

    wealth, _ = test_by_betting(X1, X2, alpha=0.01)
    big_gap.append(extend_arr(wealth, size))
    
    wealth, _ = test_by_betting(X3, X4, alpha=0.01)
    med_gap.append(extend_arr(wealth, size))
    
    wealth, _ = test_by_betting(X2, X3, alpha=0.01)
    no_gap.append(extend_arr(wealth, size))

    wealth, _ = test_by_betting(X3, X5, alpha=0.01)
    huge_gap.append(extend_arr(wealth, size))
    

plot_wealth(huge_gap, label='$\Delta=0.5$', c='purple')    
plot_wealth(big_gap, label='$\Delta=0.3$', c='navy')
plot_wealth(med_gap, label='$\Delta=0.1$', c='green')
plot_wealth(no_gap, label='$\Delta=0$', c='tab:olive')
    
plt.hlines(1/0.01, 0, size, ls='--', color='k', label='$1/\\alpha$')

plt.title('Growth of wealth process', fontsize=14)
plt.ylim(-10, 150)
plt.ylabel('Wealth')
plt.xlabel('Time')
plt.legend(bbox_to_anchor=(0.97,-0.2), ncol=3)

# Simulated Data - Figure 2

In [ ]:
def sigmoid(x): 
    return 0.5 / (1 + np.exp(-6*x))

def gen_shifted_rvs(mean1, mean2, shift): 

    X1 = bernoulli.rvs(mean1[0], size=len(mean1))
    X2 = list(bernoulli.rvs(mean1[0], size=shift))
    for t in range(shift, len(mean1)): 
        X2.append(bernoulli.rvs(p=mean2[t], size=1)[0])
        
    return X1, np.array(X2)

def gen_bernoullis(mean1, mean2): 
    # Generate bernoulli observations from given means
    
    assert len(mean1) == len(mean2)
    X1, X2 = [], []
    for t in range(len(mean1)):
        X1.append(bernoulli.rvs(p=mean1[t], size=1)[0])
        X2.append(bernoulli.rvs(p=mean2[t], size=1)[0])
        
    return np.array(X1), np.array(X2)



In [ ]:
# Smooth shift via logistic 

size = 400 
shift = 100 
mean1 = [0.3] * size # constant 
mean2 = [0.3] * shift 


for t in range(shift, size): 
    x = t - (size+shift)/2
    x /= (size - shift)/2
    mean2.append(0.3 + sigmoid(x))
    
trials = 100 

results = []
for _ in range(trials): 

    X1, X2 = gen_shifted_rvs(mean1, mean2, shift=shift)
    wealth, _ = test_by_betting(X1, X2, alpha=0.01)
    results.append(extend_arr(wealth, len(X1)))

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

mean = np.mean(results, axis=0)
std = np.std(results, axis=0)
ax2.plot(range(len(wealth)), mean, label='$\mathcal{K}_t$')
ax2.fill_between(range(len(wealth)), mean+std, mean-std, alpha=0.1)
ax2.axhline(1/0.01, ls='--', color='k', label='$1/\\alpha$')
ax2.set_ylabel('Wealth')
ax2.set_xlabel('Time')
ax2.legend(loc='upper left')
    
ax1.plot(range(size), mean1, lw=2, label='$\mu_0(t)$', c='gray')
ax1.plot(range(size), mean2, lw=2, label='$\mu_1(t)$', c='navy')
ax1.set_ylabel('$\mathbb{E}[X|\\xi_b]$')

ax1.set_ylim(0.1,0.9)
ax1.legend()


In [ ]:
# Noisy shift via sine 

shift = 100 
size = 400


mean1 = []
mean2 = [] 
for t in range(size): 
    mean1.append(np.sin(t/20)/10 + 0.4 + t/1000)
    mean2.append(np.sin(t/40)/10 + 0.4)
    
# add noise 
mean1 = np.array(mean1) + np.random.normal(0, scale=0.1, size=size)
mean1 = np.minimum(mean1, [1]*size)
mean2 = np.array(mean2) + np.random.normal(0, scale=0.1, size=size)
mean2 = np.maximum(mean2, [0]*size)

    
trials = 100 
results = []
for _ in range(trials): 

    X1, X2 = gen_bernoullis(mean1, mean2)
    wealth, _ = test_by_betting(X1, X2, alpha=0.01)
    results.append(extend_arr(wealth, len(X1)))
    

fig, (ax1, ax2) = plt.subplots(2,1, sharex=True)    
ax1.plot(range(size), mean2, lw=2, label='$\mu_0(t)$', c='gray')
ax1.plot(range(size), mean1, lw=2, label='$\mu_1(t)$', c='navy')

mean = np.mean(results, axis=0)
std = np.std(results, axis=0)
ax2.plot(range(len(wealth)), mean, label='$\mathcal{K}_t$')
ax2.fill_between(range(len(wealth)), mean+std, mean-std, alpha=0.1)
ax2.axhline(1/0.01, ls='--', color='k', label='$1/\\alpha$')
ax2.set_xlabel('Time')


# Figure 3 - Credit Loan Dataset

In [ ]:
mpl.rcParams['font.size'] = 22

cm = plt.get_cmap('viridis') 
cmap = [cm(i) for i in np.linspace(0,1,4)]

In [ ]:
# M1 - No Bonferroni  

betting_tau = load_results('betting_loan_tau')
betting_fdr = load_results('betting_loan_fdr')
perm_250_tau = load_results('perm_250_loan_tau')
perm_250_fdr = load_results('perm_250_loan_fdr')
perm_500_tau = load_results('perm_500_loan_tau')
perm_500_fdr = load_results('perm_500_loan_fdr')
perm_1000_tau = load_results('perm_1000_loan_tau')
perm_1000_fdr = load_results('perm_1000_loan_fdr')


plt.plot(np.mean(betting_tau, axis=0), np.mean(betting_fdr, axis=0), 'x', c='red')
plt.plot(np.mean(perm_250_tau, axis=0), np.mean(perm_250_fdr, axis=0), 's', c=cmap[0], label='$k=250$')
plt.plot(np.mean(perm_500_tau, axis=0), np.mean(perm_500_fdr, axis=0), 's', c=cmap[1], label='$k=500$')
plt.plot(np.mean(perm_1000_tau, axis=0), np.mean(perm_1000_fdr, axis=0), 's', c=cmap[2], label='$k=1000$')

plt.legend()
plt.xlabel('Rejection time $\\tau$ under $H_1$')
plt.ylabel('FPR under $H_0$')
plt.title('No correction (M1)')
    

In [ ]:
# M2 - With Bonferroni 

betting_tau = load_results('betting_loan_tau')
betting_fdr = load_results('betting_loan_fdr')
permb_250_tau = load_results('permb_250_loan_tau')
permb_250_fdr = load_results('permb_250_loan_fdr')
permb_500_tau = load_results('permb_500_loan_tau')
permb_500_fdr = load_results('permb_500_loan_fdr')
permb_1000_tau = load_results('permb_1000_loan_tau')
permb_1000_fdr = load_results('permb_1000_loan_fdr')


cm = plt.get_cmap('viridis') 
cmap = [cm(i) for i in np.linspace(0,1,4)]

plt.plot(np.mean(betting_tau, axis=0), np.mean(betting_fdr, axis=0), 'x',  c='red', label='Betting')
plt.plot(np.mean(permb_250_tau, axis=0), np.mean(permb_250_fdr, axis=0), 'o', c=cmap[0], label='Perm. test, $k=250$')
plt.plot(np.mean(permb_500_tau, axis=0), np.mean(permb_500_fdr, axis=0), 'o', c=cmap[1], label='Perm test, $k=500$')
plt.plot(np.mean(permb_1000_tau, axis=0), np.mean(permb_1000_fdr, axis=0), 'o', c=cmap[2], label='Perm. test, $k=1000$')

# plt.legend()
plt.xlabel('Rejection time $\\tau$ under $H_1$')
plt.ylabel('FPR under $H_0$')
plt.title('With correction (M2)')
    

In [ ]:
# FDR vs alpha 

betting_fdr = load_results('betting_loan_fdr')
perm_250_fdr = load_results('perm_250_loan_fdr')
perm_500_fdr = load_results('perm_500_loan_fdr')
perm_1000_fdr = load_results('perm_1000_loan_fdr')

alphas = np.linspace(0.005, 0.1, 20)

plt_mean_std(plt, betting_fdr, alphas, None, color='red', lw=2, ls='', marker='x')
plt_mean_std(plt, perm_250_fdr, alphas, None, marker='s', color=cmap[0])
plt_mean_std(plt, perm_500_fdr, alphas, None, marker='s', color=cmap[1])
plt_mean_std(plt, perm_1000_fdr, alphas, None, marker='s', color=cmap[2])
plt.plot([0, 0.1], [0, 0.1], color='k', lw=3, label='Desired FPR')
plt.xlabel('$\\alpha$')
plt.ylabel('False Positive Rate (FPR)')
plt.title('Type-I error')
plt.legend(fontsize=18, loc='upper left')


# Figure 3 - US Census Data 

In [ ]:
cm = plt.get_cmap('viridis') 
cmap = [cm(i) for i in np.linspace(0,1,4)]

In [ ]:
# M1 - no Bonferroni 

betting_tau = load_results('betting_census_tau')
betting_fdr = load_results('betting_census_fdr')
perm_50_tau = load_results('perm_50_census_tau')
perm_50_fdr = load_results('perm_50_census_fdr')
perm_100_tau = load_results('perm_100_census_tau')
perm_100_fdr = load_results('perm_100_census_fdr')
perm_200_tau = load_results('perm_200_census_tau')
perm_200_fdr = load_results('perm_200_census_fdr')


plt.plot(np.mean(betting_tau, axis=0), np.mean(betting_fdr, axis=0), 'x', c='red')
plt.plot(np.mean(perm_50_tau, axis=0), np.mean(perm_50_fdr, axis=0), 's', c=cmap[0], label='$k=50$')
plt.plot(np.mean(perm_100_tau, axis=0), np.mean(perm_100_fdr, axis=0), 's', c=cmap[1], label='$k=100$')
plt.plot(np.mean(perm_200_tau, axis=0), np.mean(perm_200_fdr, axis=0), 's', c=cmap[2], label='$k=200$')

plt.legend()
plt.xlabel('Rejection time $\\tau$ under $H_1$')
plt.ylabel('FPR under $H_0$')
plt.title('No correction (M1)')
    

In [ ]:
# M2 - with Bonferroni

betting_tau = load_results('betting_census_tau')
betting_fdr = load_results('betting_census_fdr')
permb_50_tau = load_results('permb_50_census_tau')
permb_50_fdr = load_results('permb_50_census_fdr')
permb_100_tau = load_results('permb_100_census_tau')
permb_100_fdr = load_results('permb_100_census_fdr')
permb_200_tau = load_results('permb_200_census_tau')
permb_200_fdr = load_results('permb_200_census_fdr')


plt.plot(np.mean(betting_tau, axis=0), np.mean(betting_fdr, axis=0), 'x', c='red', label='Betting')
plt.plot(np.mean(permb_50_tau, axis=0), np.mean(permb_50_fdr, axis=0), 'o', c=cmap[0], label='Perm. test, $k=50$')
plt.plot(np.mean(permb_100_tau, axis=0), np.mean(permb_100_fdr, axis=0), 'o', c=cmap[1], label='Perm test, $k=100$')
plt.plot(np.mean(permb_200_tau, axis=0), np.mean(permb_200_fdr, axis=0), 'o', c=cmap[2], label='Perm. test, $k=200$')

# plt.legend()
plt.xlabel('Rejection time $\\tau$ under $H_1$')
plt.ylabel('FPR under $H_0$')
plt.title('With correction (M2)')
    

In [ ]:
# FDR vs alpha 

alphas = np.linspace(0.005, 0.1, 20)

plt_mean_std(plt, betting_fdr, alphas, None, color='red', lw=2, ls='', marker='x', plot_std=False)
plt_mean_std(plt, perm_50_fdr, alphas, None, marker='s', color=cmap[0], plot_std=False)
plt_mean_std(plt, perm_100_fdr, alphas, None, marker='s', color=cmap[1], plot_std=False)
plt_mean_std(plt, perm_200_fdr, alphas, None, marker='s', color=cmap[2], plot_std=False)
plt.plot([0, 0.1], [0, 0.1], color='k', lw=3, label='Desired FPR')
plt.xlabel('$\\alpha$')
plt.ylabel('False Positive Rate (FPR)')
plt.title('Type-I error')
plt.legend(fontsize=18, loc='upper left')


# Figure 4 - Census data with distribution shift 

In [ ]:
# M1 - no Bonferroni

betting_shift = load_results('betting_shift_census')
perm_50_shift = load_results('perm_50_shift_census')
perm_100_shift = load_results('perm_100_shift_census')
perm_200_shift = load_results('perm_200_shift_census')


plt.plot(np.mean(betting_shift, axis=0), np.mean(betting_fdr, axis=0), 'x', c='red')
plt.plot(np.mean(perm_50_shift, axis=0), np.mean(perm_50_fdr, axis=0), 's', c=cmap[0], label='$k=250$')
plt.plot(np.mean(perm_100_shift, axis=0), np.mean(perm_100_fdr, axis=0), 's', c=cmap[1], label='$k=500$')
plt.plot(np.mean(perm_200_shift, axis=0), np.mean(perm_200_fdr, axis=0), 's', c=cmap[2], label='$k=1000$')

plt.legend()
plt.xlabel('Rejection time $\\tau$ under $H_1$')
plt.ylabel('FPR under $H_0$')
plt.title('No correction (M1)')
    

In [ ]:
# M2 - Bonferroni

betting_shift = load_results('betting_shift_census')
permb_50_shift = load_results('permb_50_shift_census')
permb_100_shift = load_results('permb_100_shift_census')
permb_200_shift = load_results('permb_200_shift_census')


plt.plot(np.mean(betting_shift, axis=0), np.mean(betting_fdr, axis=0), 'x', c='red')
plt.plot(np.mean(permb_50_shift, axis=0), np.mean(permb_50_fdr, axis=0), 'o', c=cmap[0], label='Perm. test, $k=250$')
plt.plot(np.mean(permb_100_shift, axis=0), np.mean(permb_100_fdr, axis=0), 'o', c=cmap[1], label='Perm test, $k=500$')
plt.plot(np.mean(permb_200_shift, axis=0), np.mean(permb_200_fdr, axis=0), 'o', c=cmap[2], label='Perm. test, $k=1000$')

# plt.legend()
plt.xlabel('Rejection time $\\tau$ under $H_1$')
plt.ylabel('FPR under $H_0$')
plt.title('With correction (M2)')
    